In [2]:
import pandas as pd
import numpy as np
import nltk

## Initial Configuration Format of Text
The initial format, at least for workability will need to be a list of words and punctuation. Getting into this format will be the first step of configuration that the text will undergo as it is simplest when trying to create a dictionary.

#### Example Text (Emma):
Using the `nltk` library (and for simplicity's sake) we will read in Jane Austen's *Emma*, which can conveniently be loaded in as a list of words.

In [17]:
print(nltk.corpus.gutenberg.fileids())
emma = nltk.corpus.gutenberg.words('austen-emma.txt')

['austen-emma.txt', 'austen-persuasion.txt', 'austen-sense.txt', 'bible-kjv.txt', 'blake-poems.txt', 'bryant-stories.txt', 'burgess-busterbrown.txt', 'carroll-alice.txt', 'chesterton-ball.txt', 'chesterton-brown.txt', 'chesterton-thursday.txt', 'edgeworth-parents.txt', 'melville-moby_dick.txt', 'milton-paradise.txt', 'shakespeare-caesar.txt', 'shakespeare-hamlet.txt', 'shakespeare-macbeth.txt', 'whitman-leaves.txt']


In [30]:
config_lists = []
for i in range(0, len(emma) - 3):
    config_lists.append(list(emma[i: i + 4]))
    

In [31]:
len(emma)

192427

In [34]:
np.array(config_lists)

array([['[', 'Emma', 'by', 'Jane'],
       ['Emma', 'by', 'Jane', 'Austen'],
       ['by', 'Jane', 'Austen', '1816'],
       ...,
       ['happiness', 'of', 'the', 'union'],
       ['of', 'the', 'union', '.'],
       ['the', 'union', '.', 'FINIS']], dtype='<U17')

'.'